<a href="https://colab.research.google.com/github/ArkS0001/Kubernetes-Project-Telaverge/blob/main/server_client_deployement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
!pip install flask requests flask-ngrok

# 1. Server Code
from flask import Flask, request, jsonify
import threading
import time
from itertools import cycle

# Define Flask server
app1 = Flask("Server 1")
app2 = Flask("Server 2")

# Simulated workers (internal sub-services)
workers = ["Worker1", "Worker2", "Worker3"]
worker_iterator = cycle(workers)

# Round-robin processing
@app1.route('/process', methods=['POST'])
def process_server1():
    data = request.get_json()
    worker = next(worker_iterator)
    return jsonify({"server": "Server 1", "worker": worker, "message": data['message']})

@app2.route('/process', methods=['POST'])
def process_server2():
    data = request.get_json()
    worker = next(worker_iterator)
    return jsonify({"server": "Server 2", "worker": worker, "message": data['message']})

# Start servers on separate threads
def start_server1():
    app1.run(port=5001)

def start_server2():
    app2.run(port=5002)

threading.Thread(target=start_server1).start()
threading.Thread(target=start_server2).start()

time.sleep(3)  # Allow servers to start
print("Servers are running...")

# 2. Client Code
import requests
import itertools

# Simulated Kubernetes service endpoints
servers = ["http://127.0.0.1:5001/process", "http://127.0.0.1:5002/process"]
server_iterator = itertools.cycle(servers)

def send_request(message):
    server = next(server_iterator)
    try:
        response = requests.post(server, json={"message": message})
        print(response.json())
    except Exception as e:
        print(f"Error connecting to {server}: {e}")

# Simulate client sending requests in a round-robin manner
print("Client is sending requests...")
for i in range(10):
    send_request(f"Request {i + 1}")
    time.sleep(1)


 * Serving Flask app 'Server 1'
 * Debug mode: off
 * Serving Flask app 'Server 2'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5002
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit


Servers are running...


INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:13:47] "POST /process HTTP/1.1" 200 -


Client is sending requests...
{'message': 'Request 1', 'server': 'Server 1', 'worker': 'Worker1'}


...

{'message': 'Request 8', 'server': 'Server 2', 'worker': 'Worker2'}


INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:13:55] "POST /process HTTP/1.1" 200 -


{'message': 'Request 9', 'server': 'Server 1', 'worker': 'Worker3'}


INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:13:56] "POST /process HTTP/1.1" 200 -


{'message': 'Request 10', 'server': 'Server 2', 'worker': 'Worker1'}


In [2]:
!pip install flask flask-limiter requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 4.0 MB/s eta 0:00:00


In [4]:
from flask import Flask, request, jsonify
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
from itertools import cycle
import threading

# Initialize Flask apps
app1 = Flask("Server 1")
app2 = Flask("Server 2")

# Rate limiter (Redis backend can be added for distributed rate limiting)
limiter1 = Limiter(get_remote_address, app=app1, default_limits=["50 per second"])
limiter2 = Limiter(get_remote_address, app=app2, default_limits=["50 per second"])

# Simulated workers
workers = ["Worker1", "Worker2", "Worker3"]
worker_iterator = cycle(workers)

# Define server endpoints
@app1.route('/process', methods=['POST'])
@limiter1.limit("50 per second")  # Rate limit for Server 1
def process_server1():
    data = request.get_json()
    worker = next(worker_iterator)
    return jsonify({"server": "Server 1", "worker": worker, "message": data['message']}), 200

@app2.route('/process', methods=['POST'])
@limiter2.limit("50 per second")  # Rate limit for Server 2
def process_server2():
    data = request.get_json()
    worker = next(worker_iterator)
    return jsonify({"server": "Server 2", "worker": worker, "message": data['message']}), 200

# Start servers on separate threads
def start_server1():
    app1.run(port=5001)

def start_server2():
    app2.run(port=5002)

threading.Thread(target=start_server1).start()
threading.Thread(target=start_server2).start()


import requests
import itertools
import threading
import time

# Server endpoints (Simulating a Load Balancer distributing requests)
servers = ["http://127.0.0.1:5001/process", "http://127.0.0.1:5002/process"]
server_iterator = itertools.cycle(servers)

# Function to send requests
def send_request(message, request_id):
    server = next(server_iterator)
    try:
        response = requests.post(server, json={"message": message})
        print(f"Request {request_id}: {response.json()}")
    except Exception as e:
        print(f"Request {request_id} failed: {e}")

# Simulate 1000 requests/second using threading
def simulate_high_throughput(total_requests, concurrency):
    threads = []
    start_time = time.time()

    for i in range(total_requests):
        t = threading.Thread(target=send_request, args=(f"Message {i+1}", i+1))
        threads.append(t)
        t.start()

        # Control concurrency to approximately match 1000 requests/sec
        if (i + 1) % concurrency == 0:
            time.sleep(1)

    for t in threads:
        t.join()

    print(f"Total time taken: {time.time() - start_time} seconds")

# Simulate 1000 requests/second
simulate_high_throughput(total_requests=1000, concurrency=100)



 * Serving Flask app 'Server 1'


INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:40] "POST /process HTTP/1.1" 200 -


 * Serving Flask app 'Server 2'
 * Debug mode: off
 * Debug mode: off


Address already in use
Port 5002 is in use by another program. Either identify and stop that program, or start the server with a different port.
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:40] "POST /process HTTP/1.1" 200 -
Address already in use
Port 5001 is in use by another program. Either identify and stop that program, or start the server with a different port.
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:40] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:40] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:40] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:40] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:40] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:40] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:40] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:40] "PO

Request 1: {'message': 'Message 1', 'server': 'Server 1', 'worker': 'Worker1'}
Request 2: {'message': 'Message 2', 'server': 'Server 2', 'worker': 'Worker2'}
Request 3: {'message': 'Message 3', 'server': 'Server 1', 'worker': 'Worker1'}
Request 4: {'message': 'Message 4', 'server': 'Server 2', 'worker': 'Worker3'}
Request 5: {'message': 'Message 5', 'server': 'Server 1', 'worker': 'Worker2'}
Request 7: {'message': 'Message 7', 'server': 'Server 1', 'worker': 'Worker3'}
Request 9: {'message': 'Message 9', 'server': 'Server 1', 'worker': 'Worker2'}
Request 6: {'message': 'Message 6', 'server': 'Server 2', 'worker': 'Worker1'}
Request 8: {'message': 'Message 8', 'server': 'Server 2', 'worker': 'Worker3'}
Request 10: {'message': 'Message 10', 'server': 'Server 2', 'worker': 'Worker1'}
Request 11: {'message': 'Message 11', 'server': 'Server 1', 'worker': 'Worker2'}
Request 13: {'message': 'Message 13', 'server': 'Server 1', 'worker': 'Worker3'}
Request 12: {'message': 'Message 12', 'server'

...

Request 949: {'message': 'Message 949', 'server': 'Server 1', 'worker': 'Worker3'}
Request 948: {'message': 'Message 948', 'server': 'Server 2', 'worker': 'Worker1'}
Request 951: {'message': 'Message 951', 'server': 'Server 1', 'worker': 'Worker2'}
Request 950: {'message': 'Message 950', 'server': 'Server 2', 'worker': 'Worker3'}
Request 952: {'message': 'Message 952', 'server': 'Server 2', 'worker': 'Worker1'}
Request 953: {'message': 'Message 953', 'server': 'Server 1', 'worker': 'Worker2'}
Request 954: {'message': 'Message 954', 'server': 'Server 2', 'worker': 'Worker3'}
Request 955: {'message': 'Message 955', 'server': 'Server 1', 'worker': 'Worker1'}
Request 957: {'message': 'Message 957', 'server': 'Server 1', 'worker': 'Worker3'}
Request 956: {'message': 'Message 956', 'server': 'Server 2', 'worker': 'Worker2'}
Request 958: {'message': 'Message 958', 'server': 'Server 2', 'worker': 'Worker1'}
Request 959: {'message': 'Message 959', 'server': 'Server 1', 'worker': 'Worker2'}
Requ

INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:57] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:57] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:57] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:57] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:57] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:57] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:57] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:57] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:57] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:57] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:57] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:57] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:12

Request 971: {'message': 'Message 971', 'server': 'Server 1', 'worker': 'Worker2'}
Request 972: {'message': 'Message 972', 'server': 'Server 2', 'worker': 'Worker3'}
Request 973: {'message': 'Message 973', 'server': 'Server 1', 'worker': 'Worker1'}
Request 975: {'message': 'Message 975', 'server': 'Server 1', 'worker': 'Worker2'}
Request 974: {'message': 'Message 974', 'server': 'Server 2', 'worker': 'Worker3'}
Request 977: {'message': 'Message 977', 'server': 'Server 1', 'worker': 'Worker1'}
Request 979: {'message': 'Message 979', 'server': 'Server 1', 'worker': 'Worker2'}
Request 976: {'message': 'Message 976', 'server': 'Server 2', 'worker': 'Worker3'}
Request 978: {'message': 'Message 978', 'server': 'Server 2', 'worker': 'Worker1'}
Request 981: {'message': 'Message 981', 'server': 'Server 1', 'worker': 'Worker2'}
Request 980: {'message': 'Message 980', 'server': 'Server 2', 'worker': 'Worker3'}
Request 983: {'message': 'Message 983', 'server': 'Server 1', 'worker': 'Worker1'}
Requ

INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:58] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jan/2025 14:15:58] "POST /process HTTP/1.1" 200 -


Request 998: {'message': 'Message 998', 'server': 'Server 2', 'worker': 'Worker2'}
Request 1000: {'message': 'Message 1000', 'server': 'Server 2', 'worker': 'Worker3'}
Request 999: {'message': 'Message 999', 'server': 'Server 1', 'worker': 'Worker1'}
Total time taken: 18.617228507995605 seconds
